In [1]:
"""
pip install matplotlib
git clone https://github.com/FlipperPA/midi-writer.git
cd midi-writer
pip install .
cd ..
jupyter notebook
"""
0

0

In [2]:
from midi_writer import MIDIFile, Notes

_ = Notes()
k = _.__dict__ #note keys
n = sorted([_.__getattribute__(i) for i in k]) #key values

In [3]:
l = n[0]  #lowwer
u = n[-1] #upper
r = u-l  #range

In [26]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Open the image
img = Image.open(r"C:\Users\原神\Downloads\image (17).jpg")
img = img.convert("L")
# Define new size (width, height)
new_size = (128, 128) 

# Resize the image
image = np.array(img.resize(new_size))
image = np.reshape(image,(*image.shape,1)).astype('float64')

In [27]:
h,w,d = image.shape #heigth,width,depth

In [28]:
#create note values by image datas
ni = []
split_value = 32
previous = 0
for i in range(previous, h, (h//split_value)):
  image_combine_height = image[i]
  for k in range(i-1,previous,-1):
      image_combine_height = image_combine_height+image[k]
  ct = image_combine_height #current image (current->ct)
  ni.append([])
  for j in range(0,w,(w//r)+1):
    ni[-1].append(ct[j])
  previous = i

In [29]:
track = len(ni)

In [30]:
#turn image rgb 3channel into 1channel (mode==sum==combine)
i1 = range(len(ni))
j1 = range(len(ni[0]))
k1 = range(len(ni[0][0]))
for i in i1:
  for j in j1:
    sum_ = 0.0
    for k in range(1):
      sum_ += ni[i][j][k]
    ni[i][j] = sum_

In [31]:
import math
mean = math.fsum(n)/len(n) #key values average

In [32]:
#scale image data into note keys
i1 = range(len(ni))
j1 = range(len(ni[0]))
for i in i1:
  smean = math.fsum([i for i in ni[i]])/len(ni[i])
  if smean==0:
      smean=1
  for j in j1:
    ni[i][j] *= mean/smean

In [33]:
#upper and lower check
i1 = range(len(ni))
j1 = range(len(ni[0]))
for i in i1:
  for j in j1:
    ni[i][j] = int(ni[i][j])
    if ni[i][j]>u:
      ni[i][j] = u
    if ni[i][j]<l:
      ni[i][j] = l

In [34]:
#basic create
channel = 0
time = 0  # In beats
duration = 1  # In beats
tempo = 60  # In BPM
velocity = 100  # 0-127, as per the MIDI standard - often used for VOLUME

MyMIDI = MIDIFile(track)
MyMIDI.add_tempo(0, time, tempo)

In [35]:
#write track==image height, pitch==image width scale value
for index, items in enumerate(ni):
    for j, pitch in enumerate(items):
        MyMIDI.add_note(index, channel, pitch, time + j, duration, velocity)

In [36]:
#output
with open("20251029124514.mid", "wb") as output_file:
    MyMIDI.write_file(output_file)